In [29]:
import os
import sys
import glob
import json
import pandas as pd
import numpy as np
import string
from snowballstemmer import TurkishStemmer
import pprint
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize 
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix




## Read JSON

In [ ]:
path = "testDatasets/*.json"
files = glob.glob(path)

ictihats = []
suc_array = []

for file in files:
    f = open(file, "r", encoding='utf-8')
    jsonData = json.loads(f.read())
    ictihats.append(jsonData['ictihat'].strip()[:20])
    suc = jsonData["Suç"] if jsonData["Suç"] != '' else "others"
    suc_array.append(suc.strip())

## Preprocessing

In [ ]:
ictihats_lower = []
for i in ictihats:
    i = i.lower()
    ictihats_lower.append(i)
    
ictihats_punctuation = []

for i in ictihats_lower:
    i = i.translate(str.maketrans("", "", string.punctuation))
    ictihats_punctuation.append(i)

## Split Dataset

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(ictihats_punctuation, 
                                                    suc_array, 
                                                    test_size=0.20,random_state=36)

## Vectorize

In [ ]:
file = open("stopwords.txt", "r", newline='', encoding='utf-8')
result = file.read()
stopwords = word_tokenize(result)

tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= stopwords)
training_data = tfidfvectorizer.fit_transform(x_train)
testing_data = tfidfvectorizer.transform(x_test)


## SVC

In [ ]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(training_data, y_train)
y_pred = svclassifier.predict(testing_data)

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

## Multinomial Naive Bayes

In [ ]:
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)

predictions = naive_bayes.predict(testing_data)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
print('Precision score: ', format(precision_score(y_test, predictions, average='macro')))
print('Recall score: ', format(recall_score(y_test, predictions ,average='macro')))
print('F1 score: ', format(f1_score(y_test, predictions ,average='macro')))

print(confusion_matrix(y_test,predictions))


In [31]:
#print(len(suc_array))
#print(ictihats_punctuation)
# index_list = []
# for i in range(0,100):
#     index_list.append(str(i))

# preprocessed_ictihats = []
# for i in ictihats_punctuation:
#        preprocessed_ictihats.append(list(filter(lambda x: x != "",i.split(" "))))

# #print(preprocessed_ictihats)
# turkStem = TurkishStemmer()
# for index, ictihat in enumerate(preprocessed_ictihats,start=0):
#     preprocessed_ictihats[index] = [turkStem.stemWord(word) for word in ictihat]
    
#print(preprocessed_ictihats)

#tfidf_wm_array
#tfidf_wm_array.shape
# frequency_matrix = pd.DataFrame(tfidf_wm_array, columns=tfidfvectorizer.get_feature_names())
# from sklearn.model_selection import train_test_split
# y = np.array([0,1,2,3,4,5,6,7,8,9,10]) 


# naive_bayes = MultinomialNB()
# naive_bayes.fit(training_data, y_train)

# predictions = naive_bayes.predict(testing_data)

# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
# print('Precision score: ', format(precision_score(y_test, predictions,
#                                            average='macro')))
# print('Recall score: ', format(recall_score(y_test, predictions
#                                            ,average='macro')))
# print('F1 score: ', format(f1_score(y_test, predictions
#                                            ,average='macro')))

# print(confusion_matrix(y_test,predictions))


# from sklearn.svm import SVC
# svclassifier = SVC(kernel='linear')
# svclassifier.fit(training_data, y_train)
# y_pred = svclassifier.predict(testing_data)

# from sklearn.metrics import classification_report, confusion_matrix
# print(confusion_matrix(y_test,y_pred))
# print(classification_report(y_test,y_pred))



# frequency_matrix
# print(tfidfvectorizer.vocabulary_)
# print(tfidf_wm.toarray())
# tfidf_tokens = tfidfvectorizer.get_feature_names()
# df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),index = index_list,columns = tfidf_tokens)
# print("\nTD-IDF Vectorizer\n")
# print(df_tfidfvect)
# vectorizer.get_feature_names_out()
# ictihat_freq = []


# for i in preprocessed_ictihats:
#     count_freq = Counter(i)
#     ictihat_freq.append(count_freq)
    
# pprint.pprint(ictihat_freq)



[[ 1  0  0  0  0  0]
 [ 0  0  1  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  1  0]
 [ 0  0  0  0  7  0]
 [ 0  0  0  0  0 10]]
                                                                                                    precision    recall  f1-score   support

                                                                                   Kasten yaralama       1.00      1.00      1.00         1
                                                    Kasten yaralama, 6136 sayılı Kanun'a muhalefet       0.00      0.00      0.00         1
Sistemi engelleme, bozma, verileri yok etme veya değiştirme, bilişim sistemine girme, cinsel taciz       0.00      0.00      0.00         0
                                                                       Uyuşturucu madde ihraç etme       0.00      0.00      0.00         1
                                                                   Uyuşturucu madde ticareti yapma       0.88      1.00      0.93         7
                                

C:\Users\onkol\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lahî'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
C:\Users\onkol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\onkol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\onkol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: Undefi